In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/wings-air/?sortby=post_date%3ADesc&pagesize=20'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    try:
        for name, rating in zip(names[1:], ratings):
            review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    except:
        review_dict[name.text] = np.nan
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    try:
        review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    
    except:
        review_dict['Overall_rating'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'wings air'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']

# Save to CSV
df.to_csv('Data/wings_air.csv',index=False)

In [3]:
# Sanity check
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
13,wings air,S Johnson,2017-06-03,✅ Verified Review | The flight from JOG to SU...,Solo Leisure,Economy Class,1,2,1,NaN,NaN,1,3,no
14,wings air,Peter Clyde,2017-03-15,❎ Unverified | Medan to Sibolga. Unfortunatel...,Solo Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no
15,wings air,F Landon,2017-02-08,✅ Verified Review | Ampana to Palu. Tobacco f...,Couple Leisure,Economy Class,2,1,1,NaN,NaN,1,2,no
16,wings air,A Peters,2016-08-04,✅ Verified Review | I flew with Wings Air (pa...,Solo Leisure,Economy Class,2,3,3,NaN,NaN,1,1,no
17,wings air,Peter van Gandt,2016-07-19,Wings Air from Denpasar to Lombok. Flight IW18...,Family Leisure,Economy Class,1,1,1,NaN,NaN,1,1,no


In [4]:
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,wings air,W Garner,2019-12-11,✅ Trip Verified | Kaimana to Manokwari. Plane...,Couple Leisure,Economy Class,7,4,4,NaN,NaN,5,4,yes
1,wings air,W Garler,2019-12-06,✅ Trip Verified | Manokwari to Kaimana. Fligh...,Couple Leisure,Economy Class,7,4,4,NaN,NaN,5,4,yes
2,wings air,W Garlen,2019-09-25,✅ Trip Verified | Lombok to Labuan Bajo. Chec...,Family Leisure,Economy Class,1,3,1,NaN,NaN,1,1,no
3,wings air,W Lee,2019-08-27,✅ Trip Verified | Wamena to Jayapura with Wing...,Solo Leisure,Economy Class,10,4,4,NaN,NaN,4,3,yes
4,wings air,W Lee,2019-08-27,✅ Trip Verified | Jayapura to Wamena with Wing...,Solo Leisure,Economy Class,8,4,4,NaN,NaN,4,3,yes
